In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Amyotrophic_Lateral_Sclerosis"
cohort = "GSE68607"

# Input paths
in_trait_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis"
in_cohort_dir = "../../input/GEO/Amyotrophic_Lateral_Sclerosis/GSE68607"

# Output paths
out_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE68607.csv"
out_gene_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE68607.csv"
out_clinical_data_file = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE68607.csv"
json_path = "../../output/preprocess/Amyotrophic_Lateral_Sclerosis/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"C9ORF72 GGGGCC expanded repeats produce splicing dysregulation which correlates with disease severity in amyotrophic lateral sclerosis [HuEx-1_0-st]"
!Series_summary	"Objective: An intronic GGGGCC-repeat expansion of C9ORF72 is the most common genetic variant of amyotrophic lateral sclerosis (ALS) and frontotemporal dementia. The mechanism of neurodegeneration is unknown, but a direct effect on RNA processing mediated by RNA foci transcribed from the repeat sequence has been proposed."
!Series_summary	"Results: Gene level analysis revealed a number of differentially expressed networks and both cell types exhibited dysregulation of a network functionally enriched for genes encoding ‘RNA splicing’ proteins.  There was a significant overlap of these genes with an independently generated list of GGGGCC-repeat protein binding partners.  At the exon level, in lymphoblastoid cells derived from C9ORF72-ALS patients splicing consistency was lower than in l

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
from typing import Optional, Callable, Dict, Any
import json
import os

# 1. Analyze gene expression data availability
is_gene_available = True  # From background info, it mentions gene expression profiling

# 2.1 Identify keys for trait, age, and gender
trait_row = 1  # "patient group" contains ALS status
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Define conversion functions
def convert_trait(value: str) -> Optional[int]:
    """Convert ALS status to binary value."""
    if value is None:
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if 'control' in value.lower():
        return 0
    elif 'als' in value.lower():
        return 1
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age to float, but we don't have age data."""
    return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender to binary value, but we don't have gender data."""
    return None

# 3. Save metadata for initial filtering
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Extract clinical features if available
if trait_row is not None:
    # Load the clinical data (assuming it was loaded in previous steps)
    clinical_data = pd.DataFrame({i: values for i, values in {0: ['subject id: Control1', 'subject id: Control2', 'subject id: Control3', 'subject id: Control4', 'subject id: Control5', 'subject id: Control6', 'subject id: Control7', 'subject id: Control8', 'subject id: Control9', 'subject id: Control10', 'subject id: Control11', 'subject id: Control12', 'subject id: Control13', 'subject id: Control14', 'subject id: Control15', 'subject id: Patient1', 'subject id: Patient2', 'subject id: Patient3', 'subject id: Patient4', 'subject id: Patient5', 'subject id: Patient6', 'subject id: Patient7', 'subject id: Patient8', 'subject id: Patient9', 'subject id: Patient10', 'subject id: Patient11', 'subject id: Patient12', 'subject id: Patient13', 'subject id: Patient14', 'subject id: Patient15'], 1: ['patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: Control', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72', 'patient group: ALS due to mtC9ORF72'], 2: ['cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells', 'cell type: Cultured lymphoblastoid cells']}.items()})
    
    # Extract and process clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the processed clinical data
    print("Preview of selected clinical features:")
    print(preview_df(selected_clinical_df))
    
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save clinical data
    selected_clinical_df.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of selected clinical features:
{0: [0.0], 1: [0.0], 2: [nan]}
Clinical data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/clinical_data/GSE68607.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['ENST00000000233', 'ENST00000000412', 'ENST00000000442',
       'ENST00000001008', 'ENST00000001146', 'ENST00000002125',
       'ENST00000002165', 'ENST00000002501', 'ENST00000002596',
       'ENST00000002829', 'ENST00000003084', 'ENST00000003100',
       'ENST00000003302', 'ENST00000003583', 'ENST00000003607',
       'ENST00000003834', 'ENST00000003912', 'ENST00000004103',
       'ENST00000004531', 'ENST00000004921'],
      dtype='object', name='ID')

Gene data dimensions: 121741 genes × 69 samples


### Step 4: Gene Identifier Review

In [5]:
# Looking at the gene identifiers, these are ENST identifiers which represent Ensembl transcript IDs,
# not standard human gene symbols. These will need to be mapped to gene symbols for consistent analysis.

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['ENST00000456328', 'ENST00000450305', 'ENST00000438504', 'ENST00000423562', 'ENST00000488147'], 'transcript_symbol': ['DDX11L10-202', 'DDX11L10-201', 'WASH5P-203', 'WASH5P-201', 'WASH5P-204'], 'chromosome': ['1', '1', '1', '1', '1'], 'band': ['p36.33', 'p36.33', 'p36.33', 'p36.33', 'p36.33'], 'start_position': [11874.0, 12010.0, 14363.0, 14363.0, 14404.0], 'end_position': [14412.0, 13670.0, 29370.0, 29370.0, 29570.0], 'SPOT_ID': ['ENSG00000223972', 'ENSG00000223972', 'ENSG00000227232', 'ENSG00000227232', 'ENSG00000227232'], 'ORF': ['DDX11L10', 'DDX11L10', 'WASH5P', 'WASH5P', 'WASH5P'], 'biotype': ['protein_coding', 'protein_coding', 'protein_coding', 'protein_coding', 'protein_coding'], 'gene_description': ['DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 10 [Source:HGNC Symbol;Acc:14125]', 'DEAD/H (Asp-Glu-Ala-Asp/His) box polypeptide 11 like 10 [Source:HGNC Symbol;Acc:14125]', 'WAS protein family homolog 5 pseudogene [Source:HGNC Symbol;Acc:33884

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Based on the preview, I can see:
# The gene expression data uses 'ID' column with ENST identifiers (Ensembl transcript IDs)
# The gene annotation data has 'ID' column matching these transcript IDs
# The 'ORF' column appears to contain gene symbols

# 2. Create gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, "ID", "ORF")
print("Gene mapping (first few rows):")
print(gene_mapping.head())
print(f"Number of mappings: {len(gene_mapping)}")

# 3. Apply gene mapping to convert probe-level measurements to gene-level expression data
# This handles many-to-many mappings between probes and genes
gene_data = apply_gene_mapping(gene_data, gene_mapping)
print("Gene expression data after mapping:")
print(f"Shape: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")
print("First few gene symbols:")
print(gene_data.index[:5])

# Save processed gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene mapping (first few rows):
                ID      Gene
0  ENST00000456328  DDX11L10
1  ENST00000450305  DDX11L10
2  ENST00000438504    WASH5P
3  ENST00000423562    WASH5P
4  ENST00000488147    WASH5P
Number of mappings: 134266


Gene expression data after mapping:
Shape: 28998 genes × 69 samples
First few gene symbols:
Index(['A1BG', 'A1CF', 'A26C1B', 'A2BP1', 'A2LD1'], dtype='object', name='Gene')


Gene expression data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE68607.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the index of gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"First 5 gene symbols after normalization: {normalized_gene_data.index[:5]}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene data saved to {out_gene_data_file}")

# 2. Check if clinical data was properly loaded
# First, reload the clinical_data to make sure we're using the original data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Print the sample IDs to understand the data structure
print("Sample IDs in clinical data:")
print(clinical_data.columns[:5], "...")  # Show first 5 sample IDs

# Print the sample IDs in gene expression data
print("Sample IDs in gene expression data:")
print(normalized_gene_data.columns[:5], "...")  # Show first 5 sample IDs

# Extract clinical features using the actual sample IDs
is_trait_available = trait_row is not None
linked_data = None

if is_trait_available:
    # Extract clinical features with proper sample IDs
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    print(f"Clinical data preview: {preview_df(selected_clinical_df, n=3)}")
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    # Make sure both dataframes have compatible indices/columns
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before handling missing values: {linked_data.shape}")
    
    if linked_data.shape[0] == 0:
        print("WARNING: No samples matched between clinical and genetic data!")
        # Create a sample dataset for demonstration
        print("Using gene data with artificial trait values for demonstration")
        is_trait_available = False
        is_biased = True
        linked_data = pd.DataFrame(index=normalized_gene_data.columns)
        linked_data[trait] = 1  # Placeholder
    else:
        # 3. Handle missing values
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
        
        # 4. Determine if trait and demographic features are biased
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
else:
    print("Trait data was determined to be unavailable in previous steps.")
    is_biased = True  # Set to True since we can't evaluate without trait data
    linked_data = pd.DataFrame(index=normalized_gene_data.columns)
    linked_data[trait] = 1  # Add a placeholder trait column
    print(f"Using placeholder data due to missing trait information, shape: {linked_data.shape}")

# 5. Validate and save cohort info
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data,
    note="Dataset contains gene expression data from multiple sclerosis patients, but there were issues linking clinical and genetic data."
)

# 6. Save linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associational studies.")

Gene data shape after normalization: (19964, 69)
First 5 gene symbols after normalization: Index(['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2ML1'], dtype='object', name='Gene')


Normalized gene data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/gene_data/GSE68607.csv


Sample IDs in clinical data:
Index(['!Sample_geo_accession', 'GSM1677001', 'GSM1677002', 'GSM1677003',
       'GSM1677004'],
      dtype='object') ...
Sample IDs in gene expression data:
Index(['GSM1677001', 'GSM1677002', 'GSM1677003', 'GSM1677004', 'GSM1677005'], dtype='object') ...
Clinical data shape: (1, 69)
Clinical data preview: {'GSM1677001': [0.0], 'GSM1677002': [0.0], 'GSM1677003': [0.0], 'GSM1677004': [0.0], 'GSM1677005': [0.0], 'GSM1677006': [0.0], 'GSM1677007': [0.0], 'GSM1677008': [0.0], 'GSM1677009': [0.0], 'GSM1677010': [0.0], 'GSM1677011': [0.0], 'GSM1677012': [0.0], 'GSM1677013': [0.0], 'GSM1677014': [0.0], 'GSM1677015': [0.0], 'GSM1677016': [1.0], 'GSM1677017': [1.0], 'GSM1677018': [1.0], 'GSM1677019': [1.0], 'GSM1677020': [1.0], 'GSM1677021': [1.0], 'GSM1677022': [1.0], 'GSM1677023': [1.0], 'GSM1677024': [1.0], 'GSM1677025': [1.0], 'GSM1677026': [1.0], 'GSM1677027': [1.0], 'GSM1677028': [1.0], 'GSM1677029': [1.0], 'GSM1677030': [1.0], 'GSM1677031': [1.0], 'GSM1677032

Data shape after handling missing values: (69, 19965)
For the feature 'Amyotrophic_Lateral_Sclerosis', the least common label is '0.0' with 15 occurrences. This represents 21.74% of the dataset.
The distribution of the feature 'Amyotrophic_Lateral_Sclerosis' in this dataset is fine.

Data shape after removing biased features: (69, 19965)


Linked data saved to ../../output/preprocess/Amyotrophic_Lateral_Sclerosis/GSE68607.csv
